**importing the dataset files**

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c sound-based-drone-fault-classification-using-multi

100% 6.96G/6.97G [00:48<00:00, 195MB/s]
100% 6.97G/6.97G [00:48<00:00, 153MB/s]


In [ ]:
!unzip "/content/sound-based-drone-fault-classification-using-multi.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_640_EoeunHill_864_snr=10.9312525240046.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_641_SportsComplex_342_snr=13.040822347725339.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_642_DuckPond_767_snr=10.161167144259057.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_643_DuckPond_754_snr=14.698899950049299.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_644_ConstructionSite_609_snr=13.773810348372994.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_645_ConstructionSite_784_snr=10.05008555945102.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_646_DuckPond_379_snr=12.583003612640931.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_647_WestDoor_234_snr=11.418478588496932.wav  
  inflating: Dataset/Dataset/drone_C/C/valid/mic2/C_F_MF3_648_DuckPond_141_snr=10.037282510476313.

**building the training dataset**

In [ ]:
def feature_extractor(audio_file):
    features = []

    audio, sample_rate = librosa.load(audio_file, sr=None)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=48)#32 #40

    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

    features.append(mfccs_scaled_features)

    return np.array(features)


In [ ]:
def parse_filename(filename):

    file_name = filename.replace("mic1", "").replace("mic2", "")

    components = filename.split('_')
    model_type = components[0]
    maneuvering_direction = components[1]
    fault = components[2]

    return file_name, model_type, maneuvering_direction, fault


**The following section marks the beginnning of preparation of the trainset**

In each code block I have tried to parse the file name for each drone i.e A,B,C and then creating a pandas dataframe which could then be used for passing into the Neural Net

In [ ]:
data=[]
import os
import librosa
import numpy as np
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_A/A/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/train/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)

        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))



In [ ]:
import pandas as pd
extracted_feat=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])

In [ ]:
data1=[]
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_A/A/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/train/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data1.append((file_name,mic2_features, model_type, maneuvering_direction, fault))



In [ ]:
import pandas as pd
extracted_feat1=pd.DataFrame(data1,columns=['file_name','feature','model','manev_dir','fault'])


**combined dataframe of drone A and has features of both mic1 and mic2 in the same format as of submission doc**

In [ ]:
merge=pd.concat((extracted_feat,extracted_feat1),axis=0)
merge

,file_name,feature,model,manev_dir,fault
0,A_R_PC4__181_DuckPond_832_snr=12.9990634975220...,"[[-27.411182, 51.13649, 17.127937, 14.011588, ...",A,R,PC4
1,A_B_PC4__70_ConstructionSite_883_snr=14.300041...,"[[-0.94964373, 48.757664, 3.3577995, 24.484795...",A,B,PC4
2,A_CC_PC1__497_DuckPond_315_snr=14.513820922809...,"[[-8.810595, 45.18904, 21.577, 19.322643, -3.2...",A,CC,PC1
3,A_R_PC2__605_EoeunHill_696_snr=13.066321185570...,"[[-29.069672, 76.73343, 23.048048, 20.673397, ...",A,R,PC2
4,A_CC_N__292_EoeunHill_218_snr=11.5380817003484...,"[[-33.643623, 56.918404, 20.640387, 18.375624,...",A,CC,N
...,...,...,...,...,...
32395,A_R_MF3__554_WestDoor_93_snr=10.60652176113135...,"[[-1.311041, 49.847, 21.315643, 12.360919, -6....",A,R,MF3
32396,A_CC_MF1__473_ConstructionSite_667_snr=14.7711...,"[[-44.602577, 19.311737, 9.538766, 12.86322, -...",A,CC,MF1
32397,A_C_PC2__132_SportsComplex_31_snr=13.697376285...,"[[-12.253685, 51.89907, 15.412756, 14.089094, ...",A,C,PC2
32398,A_CC_PC1__192_WestDoor_723_snr=13.754568852770...,"[[-11.190527, 53.802483, 16.824379, 12.444004,...",A,CC,PC1


In [ ]:
data=[]
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_B/B/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/train/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))



In [ ]:
import pandas as pd
extracted_feat2=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_B/B/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/train/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))



In [ ]:
import pandas as pd
extracted_feat3=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
merge1=pd.concat((extracted_feat2,extracted_feat3),axis=0)
merge1

,file_name,feature,model,manev_dir,fault
0,B_C_MF4__280_WestDoor_559_snr=10.9446086478422...,"[[-33.583065, 58.372337, 20.295704, 13.051477,...",B,C,MF4
1,B_F_MF3__560_EoeunHill_744_snr=11.725480095744...,"[[-63.094753, 71.90993, 33.72282, 20.790226, 7...",B,F,MF3
2,B_CC_PC1__236_EoeunHill_256_snr=10.87606161272...,"[[-28.986244, 81.493866, 27.147274, 13.330821,...",B,CC,PC1
3,B_B_PC2__913_EoeunHill_135_snr=11.051761515750...,"[[-29.1241, 72.82776, 22.429396, 15.921038, -8...",B,B,PC2
4,B_CC_MF2__703_ConstructionSite_182_snr=12.2049...,"[[-57.439217, 51.37913, 17.938232, 11.694244, ...",B,CC,MF2
...,...,...,...,...,...
32395,B_R_PC3__579_EoeunHill_810_snr=14.504628019433...,"[[-52.69935, 61.55259, 16.29541, 17.576885, -7...",B,R,PC3
32396,B_F_MF3__919_EoeunHill_672_snr=13.052828027927...,"[[-45.858856, 83.546844, 21.753172, 10.808159,...",B,F,MF3
32397,B_F_PC4__888_SportsComplex_634_snr=12.43935483...,"[[-19.001219, 56.485527, 12.27121, 17.82978, 2...",B,F,PC4
32398,B_C_N__159_SportsComplex_85_snr=12.18211270349...,"[[-9.386095, 56.374344, 2.029418, 7.96591, 1.3...",B,C,N


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_C/C/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/train/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr_feat4=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_C/C/train/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/train/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name +'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr_feat5=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
merge2=pd.concat((extr_feat4,extr_feat5),axis=0)
merge2

,file_name,feature,model,manev_dir,fault
0,C_B_PC1__180_WestDoor_96_snr=13.24034066475225...,"[[-25.194973, 57.50477, 20.498966, 11.396554, ...",C,B,PC1
1,C_R_PC2__821_DuckPond_276_snr=13.8555239417532...,"[[-44.11828, 53.735794, 22.481426, 22.403082, ...",C,R,PC2
2,C_B_PC2__228_SportsComplex_1095_snr=12.5220425...,"[[-15.202443, 49.57444, 13.440121, 20.574936, ...",C,B,PC2
3,C_B_PC3__85_WestDoor_112_snr=12.24399729529949...,"[[-34.45265, 63.16901, 17.37912, 12.539871, 5....",C,B,PC3
4,C_CC_PC3__548_ConstructionSite_760_snr=11.8710...,"[[-28.106152, 59.974243, 14.5490885, 15.273798...",C,CC,PC3
...,...,...,...,...,...
32395,C_B_MF1__233_SportsComplex_849_snr=10.81522907...,"[[-40.306015, 41.307163, 25.613792, 22.343899,...",C,B,MF1
32396,C_L_MF1__536_EoeunHill_6_snr=11.28907077340880...,"[[-50.557198, 39.579926, 33.585426, 22.792835,...",C,L,MF1
32397,C_C_N__159_EoeunHill_647_snr=11.19960371220132...,"[[-70.76834, 45.974228, 31.089773, 20.048328, ...",C,C,N
32398,C_CC_MF3__259_ConstructionSite_104_snr=14.2177...,"[[-58.532223, 44.36705, 15.318425, 22.839355, ...",C,CC,MF3


In [ ]:
final_df=pd.concat([merge,merge1,merge2],axis=0)
final_df

,file_name,feature,model,manev_dir,fault
0,A_R_PC4__181_DuckPond_832_snr=12.9990634975220...,"[[-27.411182, 51.13649, 17.127937, 14.011588, ...",A,R,PC4
1,A_B_PC4__70_ConstructionSite_883_snr=14.300041...,"[[-0.94964373, 48.757664, 3.3577995, 24.484795...",A,B,PC4
2,A_CC_PC1__497_DuckPond_315_snr=14.513820922809...,"[[-8.810595, 45.18904, 21.577, 19.322643, -3.2...",A,CC,PC1
3,A_R_PC2__605_EoeunHill_696_snr=13.066321185570...,"[[-29.069672, 76.73343, 23.048048, 20.673397, ...",A,R,PC2
4,A_CC_N__292_EoeunHill_218_snr=11.5380817003484...,"[[-33.643623, 56.918404, 20.640387, 18.375624,...",A,CC,N
...,...,...,...,...,...
32395,C_B_MF1__233_SportsComplex_849_snr=10.81522907...,"[[-40.306015, 41.307163, 25.613792, 22.343899,...",C,B,MF1
32396,C_L_MF1__536_EoeunHill_6_snr=11.28907077340880...,"[[-50.557198, 39.579926, 33.585426, 22.792835,...",C,L,MF1
32397,C_C_N__159_EoeunHill_647_snr=11.19960371220132...,"[[-70.76834, 45.974228, 31.089773, 20.048328, ...",C,C,N
32398,C_CC_MF3__259_ConstructionSite_104_snr=14.2177...,"[[-58.532223, 44.36705, 15.318425, 22.839355, ...",C,CC,MF3


In [ ]:
features = final_df['feature'].tolist()
labels_status = final_df['fault'].tolist()
labels_direction = final_df['manev_dir'].tolist()
labels_model = final_df['model'].tolist()

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
labels_status1= label_encoder.fit_transform(labels_status)

In [ ]:
labels_status1

array([8, 8, 5, ..., 4, 2, 2])

In [ ]:
label_encoder1 = preprocessing.LabelEncoder()
labels_direction1= label_encoder1.fit_transform(labels_direction)
labels_direction1

array([5, 0, 2, ..., 1, 2, 3])

In [ ]:
label_encoder2 = preprocessing.LabelEncoder()
labels_model1=label_encoder2.fit_transform(labels_model)
labels_model1

array([0, 0, 0, ..., 2, 2, 2])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
X_train_tensor = torch.tensor(features, dtype=torch.float32)
y_train_status_tensor = torch.tensor(labels_status1, dtype=torch.long)
y_train_direction_tensor = torch.tensor(labels_direction1, dtype=torch.long)
y_train_model_tensor = torch.tensor(labels_model1, dtype=torch.long)

<ipython-input-29-ddc668828482>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  X_train_tensor = torch.tensor(features, dtype=torch.float32)


In [ ]:
X_train_tensor.shape

torch.Size([194400, 1, 48])

In [ ]:
# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_status_tensor, y_train_direction_tensor,y_train_model_tensor)
#val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_status_tensor, y_val_direction_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
num_epochs = 40
input_size = 1
hidden_size = 768  #512 #640 #1280 #320
num_classes_status = 9
num_classes_direction = 6
num_classes_model = 3

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class MultitaskDroneNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes_status, num_classes_direction):
        super(MultitaskDroneNN, self).__init__()
        # Feature extractor layers
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, padding='same')
        self.bn1 = nn.BatchNorm1d(num_features=64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        ######## this is the second set of layer
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding='same')
        self.bn2 = nn.BatchNorm1d(num_features=128)
        self.relu = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        ####### this is the third set of layer
        self.conv3=nn.Conv1d(in_channels=128,out_channels=256,kernel_size=3,padding='same')
        self.bn3=nn.BatchNorm1d(256)
        self.relu2=nn.ReLU()
        self.pool3=nn.MaxPool1d(kernel_size=2)
        ########this is the fourth set of layers
        self.conv4=nn.Conv1d(in_channels=256,out_channels=128,kernel_size=3,padding='same')# added this layer in my this attempt only
        self.bn4=nn.BatchNorm1d(128)# just added
        self.relu3=nn.ReLU()#just added

        # Status classifier
        self.status_class = nn.Linear(hidden_size, 128)
        self.drop1=nn.Dropout(0.4)
        self.status_classifier = nn.Linear(128, num_classes_status)
        #model classifier
        self.model_classsifier=nn.Linear(hidden_size,num_classes_model)
        # Direction classifier
        self.direction_classifier1 = nn.Linear(hidden_size, 128)
        self.drop1=nn.Dropout(0.4)
        self.direction_classifier2 = nn.Linear(128, num_classes_direction)

    def forward(self, x):
        # Feature extraction
        x = self.conv1(x)
        x=self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x=self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x=self.conv3(x)
        x=self.bn3(x)
        x=self.relu2(x)
        x=self.pool3(x)
        x=self.conv4(x)
        x=self.bn4(x)
        x=self.relu3(x)

        x = torch.flatten(x, 1)

        # Task-specific classifiers
        status_output = self.status_class(x)
        status_output=self.drop1(status_output)
        status_output=self.status_classifier(status_output)
        ##################################################
        direction_output1 = self.direction_classifier1(x)
        direction_output2=self.drop1(direction_output1)
        direction_output3=self.direction_classifier2(direction_output2)
        ##################################################
        model_output=self.model_classsifier(x)
        #################################################
        return status_output, direction_output3,model_output

# Instantiate the model
model = MultitaskDroneNN(input_size, hidden_size, num_classes_status, num_classes_direction)
print(model)


MultitaskDroneNN(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=same)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=same)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (sta

In [ ]:
model = MultitaskDroneNN(input_size, hidden_size, num_classes_status, num_classes_direction)
criterion_status = nn.CrossEntropyLoss()
criterion_direction = nn.CrossEntropyLoss()
criterion_model = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to the device
model.to(device)
for epoch in range(num_epochs):
    # Training
    model.train()
    total_loss_status = 0.0
    total_loss_direction = 0.0
    total_loss_model=0.0
    for inputs, targets_status, targets_direction,targets_model in train_loader:
        # Move data to the device
        inputs, targets_status, targets_direction,targets_model = inputs.to(device), targets_status.to(device), targets_direction.to(device),targets_model.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs_status, outputs_direction,outputs_model = model(inputs)

        # Compute the loss for each task
        loss_status = criterion_status(outputs_status, targets_status)
        loss_direction = criterion_direction(outputs_direction, targets_direction)
        loss_model=criterion_model(outputs_model,targets_model)
        total_loss = loss_status + loss_direction+loss_model
        total_loss.backward()
        optimizer.step()


        total_loss_status += loss_status.item() * inputs.size(0)
        total_loss_direction += loss_direction.item() * inputs.size(0)
        total_loss_model += loss_model.item() * inputs.size(0)

    avg_loss_status = total_loss_status / len(train_loader.dataset)
    avg_loss_direction = total_loss_direction / len(train_loader.dataset)
    avg_loss_model = total_loss_model / len(train_loader.dataset)
    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss (Status): {avg_loss_status:.4f}, "
          f"Train Loss (Direction): {avg_loss_direction:.4f},"
          f"Train loss(model):{avg_loss_model:.4f}")

Epoch [1/40], Train Loss (Status): 0.6463, Train Loss (Direction): 0.9016,Train loss(model):0.0272
Epoch [2/40], Train Loss (Status): 0.4025, Train Loss (Direction): 0.5888,Train loss(model):0.0106
Epoch [3/40], Train Loss (Status): 0.3369, Train Loss (Direction): 0.4944,Train loss(model):0.0081
Epoch [4/40], Train Loss (Status): 0.2975, Train Loss (Direction): 0.4401,Train loss(model):0.0065
Epoch [5/40], Train Loss (Status): 0.2705, Train Loss (Direction): 0.4009,Train loss(model):0.0057
Epoch [6/40], Train Loss (Status): 0.2494, Train Loss (Direction): 0.3722,Train loss(model):0.0049
Epoch [7/40], Train Loss (Status): 0.2344, Train Loss (Direction): 0.3474,Train loss(model):0.0044
Epoch [8/40], Train Loss (Status): 0.2162, Train Loss (Direction): 0.3263,Train loss(model):0.0041
Epoch [9/40], Train Loss (Status): 0.2058, Train Loss (Direction): 0.3089,Train loss(model):0.0038
Epoch [10/40], Train Loss (Status): 0.1974, Train Loss (Direction): 0.2950,Train loss(model):0.0034
Epoch [11

**Validation dataset**

In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_A/A/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/valid/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr1=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])

In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_A/A/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/valid/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr2=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
mer2=pd.concat((extr1,extr2),axis=0)


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_B/B/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/valid/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr3=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
data=[]
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_B/B/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/valid/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr4=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
mer3=pd.concat((extr3,extr4),axis=0)


In [ ]:
data=[]
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_C/C/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/valid/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr5=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
data=[]
# Directories containing audio files for mic1 and mic2
mic1_folder = "/content/Dataset/Dataset/drone_C/C/valid/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/valid/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr6=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
mer4=pd.concat((extr5,extr6),axis=0)


In [ ]:
mer4

,file_name,feature,model,manev_dir,fault
0,C_F_N_669_DuckPond_418_snr=13.352893625702619....,"[[-66.861275, 57.255024, 31.764381, 26.951836,...",C,F,N
1,C_B_MF4_691_ConstructionSite_385_snr=11.005232...,"[[-30.463121, 44.12012, 19.393333, 12.968475, ...",C,B,MF4
2,C_L_MF4_64_DuckPond_22_snr=14.848397617849976....,"[[-71.6208, 46.500244, 25.586826, 25.533451, 5...",C,L,MF4
3,C_B_MF2_728_ConstructionSite_858_snr=14.038371...,"[[-45.818527, 52.28072, 28.120922, 19.279552, ...",C,B,MF2
4,C_CC_MF2_796_WestDoor_15_snr=12.61608391006143...,"[[-57.41992, 47.331497, 33.612892, 23.299152, ...",C,CC,MF2
...,...,...,...,...,...
10795,C_B_MF4_744_SportsComplex_1063_snr=14.96571842...,"[[-43.904663, 48.278263, 14.644419, 15.52396, ...",C,B,MF4
10796,C_B_PC4_650_SportsComplex_450_snr=14.061618359...,"[[-36.72683, 55.271797, 14.976109, 11.123644, ...",C,B,PC4
10797,C_B_PC1_77_DuckPond_144_snr=13.959394772177815...,"[[-29.202799, 56.638935, 24.873816, 14.368211,...",C,B,PC1
10798,C_C_N_740_EoeunHill_65_snr=11.74167846941272.w...,"[[-32.125214, 58.93247, 19.985088, 10.567224, ...",C,C,N


In [ ]:
final=pd.concat([mer2,mer3,mer4],axis=0)
final

,file_name,feature,model,manev_dir,fault
0,A_F_PC1_716_ConstructionSite_854_snr=11.009498...,"[[-29.256252, 54.679234, 18.430553, 21.218914,...",A,F,PC1
1,A_C_PC1_791_EoeunHill_866_snr=10.8792751883201...,"[[-25.946657, 64.20767, 32.8358, 23.054546, -4...",A,C,PC1
2,A_L_PC4_640_SportsComplex_1026_snr=11.76094319...,"[[-8.464891, 60.428955, 6.9776025, 7.469012, -...",A,L,PC4
3,A_CC_MF2_812_SportsComplex_542_snr=11.74514581...,"[[8.316191, 44.214577, 9.174343, 6.7463226, -4...",A,CC,MF2
4,A_L_PC2_645_WestDoor_661_snr=14.65706054481551...,"[[-41.91748, 69.68642, 25.139477, 11.808891, 0...",A,L,PC2
...,...,...,...,...,...
10795,C_B_MF4_744_SportsComplex_1063_snr=14.96571842...,"[[-43.904663, 48.278263, 14.644419, 15.52396, ...",C,B,MF4
10796,C_B_PC4_650_SportsComplex_450_snr=14.061618359...,"[[-36.72683, 55.271797, 14.976109, 11.123644, ...",C,B,PC4
10797,C_B_PC1_77_DuckPond_144_snr=13.959394772177815...,"[[-29.202799, 56.638935, 24.873816, 14.368211,...",C,B,PC1
10798,C_C_N_740_EoeunHill_65_snr=11.74167846941272.w...,"[[-32.125214, 58.93247, 19.985088, 10.567224, ...",C,C,N


In [ ]:
feat_mic1 = final['feature'].tolist()
label_status = final['fault'].tolist()
label_direction = final['manev_dir'].tolist()
label_model=final['model']

In [ ]:
label_status1= label_encoder.fit_transform(label_status)
label_status1

array([5, 5, 8, ..., 5, 4, 3])

In [ ]:
label_direction1= label_encoder1.fit_transform(label_direction)
label_direction1

array([3, 1, 4, ..., 0, 1, 3])

In [ ]:
label_model1=label_encoder2.fit_transform(label_model)
label_model1

array([0, 0, 0, ..., 2, 2, 2])

In [ ]:
X9=torch.tensor(feat_mic1,dtype=torch.float32)

In [ ]:
y_val_status_tensor = torch.tensor(label_status1, dtype=torch.long)
y_val_direction_tensor = torch.tensor(label_direction1, dtype=torch.long)
y_val_model_tensor = torch.tensor(label_model1, dtype=torch.long)

In [ ]:
val_dataset = torch.utils.data.TensorDataset(X9, y_val_status_tensor, y_val_direction_tensor,y_val_model_tensor )
batch_size = 32
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
model.eval()
with torch.no_grad():
        total_correct_status = 0
        total_correct_direction = 0
        total_correct_model=0
        for inputs, targets_status, targets_direction,targets_model in val_loader:
            inputs, targets_status, targets_direction,targets_model = inputs.to(device), targets_status.to(device), targets_direction.to(device),targets_model.to(device)
            outputs_status, outputs_direction,outputs_model = model(inputs)

            # Compute accuracy for each task
            _, predicted_status = torch.max(outputs_status, 1)
            _, predicted_direction = torch.max(outputs_direction, 1)
            _, predicted_model = torch.max(outputs_model, 1)
            total_correct_status += (predicted_status == targets_status).sum().item()
            total_correct_direction += (predicted_direction == targets_direction).sum().item()
            total_correct_model += (predicted_model == targets_model).sum().item()

        # Compute average validation accuracy
        accuracy_status = total_correct_status / len(val_loader.dataset)
        accuracy_direction = total_correct_direction / len(val_loader.dataset)
        accuracy_model=total_correct_model / len(val_loader.dataset)
        print(f"Val Accuracy (Status): {accuracy_status:.4f}, "
          f"Val Accuracy (Direction): {accuracy_direction:.4f},"
          f"Val Accuracy (model): {accuracy_model:.4f},")

Val Accuracy (Status): 0.9195, Val Accuracy (Direction): 0.8841,Val Accuracy (model): 0.9980,


In [ ]:
model.eval()

MultitaskDroneNN(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=same)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=same)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (sta

**building the test submission**

In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_A/A/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/test/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr7=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_A/A/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_A/A/test/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr8=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
mer8=pd.concat((extr8,extr7),axis=0)


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_B/B/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/test/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr9=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_B/B/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_B/B/test/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr10=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])

In [ ]:
mer9=pd.concat((extr9,extr10),axis=0)


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_C/C/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/test/mic2"

for filename in os.listdir(mic1_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic1_folder, filename)
        mic1_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic1'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic1_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr11=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
# Directories containing audio files for mic1 and mic2
data=[]
mic1_folder = "/content/Dataset/Dataset/drone_C/C/test/mic1"
mic2_folder = "/content/Dataset/Dataset/drone_C/C/test/mic2"

for filename in os.listdir(mic2_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(mic2_folder, filename)
        mic2_features = feature_extractor(file_path)
        file_name,model_type, maneuvering_direction, fault = parse_filename(filename)
        file_name=file_name+'_mic2'
        # Append mic1 features and corresponding labels to the list
        data.append((file_name,mic2_features, model_type, maneuvering_direction, fault))

In [ ]:
import pandas as pd
extr12=pd.DataFrame(data,columns=['file_name','feature','model','manev_dir','fault'])


In [ ]:
mer10=pd.concat((extr11,extr12),axis=0)


In [ ]:
final_test=pd.concat([mer8,mer9,mer10],axis=0)
final_test

,file_name,feature,model,manev_dir,fault
0,A_R_PC2_17_ConstructionSite_104_snr=11.7466147...,"[[-16.205215, 53.584282, 18.09502, 23.578316, ...",A,R,PC2
1,A_B_MF3_881_EoeunHill_369_snr=14.2721269545175...,"[[-39.81353, 38.202774, 18.724974, 8.9491205, ...",A,B,MF3
2,A_C_MF2_956_ConstructionSite_493_snr=14.999665...,"[[-36.244408, 42.683, 20.310837, 17.933365, -1...",A,C,MF2
3,A_C_MF1_871_WestDoor_150_snr=14.11904469828211...,"[[-27.275352, 38.162643, 22.609488, 23.236496,...",A,C,MF1
4,A_R_MF2_117_EoeunHill_258_snr=12.3358923008580...,"[[-14.087541, 48.925194, 13.475619, 5.610791, ...",A,R,MF2
...,...,...,...,...,...
10795,C_L_MF4_921_DuckPond_408_snr=14.68987604267104...,"[[-66.324585, 41.434002, 19.623295, 18.291168,...",C,L,MF4
10796,C_B_MF2_822_EoeunHill_722_snr=11.7196268001049...,"[[-43.152386, 60.153015, 35.06806, 16.570435, ...",C,B,MF2
10797,C_B_N_929_WestDoor_551_snr=14.21089595681281.w...,"[[-55.298943, 30.89442, 16.506996, 17.082054, ...",C,B,N
10798,C_CC_N_887_EoeunHill_650_snr=12.16161579225678...,"[[-38.6005, 58.68376, 26.395235, 13.141326, -0...",C,CC,N


In [ ]:
feat_test=final_test['feature'].tolist()
feat_test=np.array(feat_test)

In [ ]:
feat_test=torch.tensor(feat_test)

In [ ]:
test_dataset = torch.utils.data.TensorDataset(feat_test )
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
model.to(device)

MultitaskDroneNN(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=same)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=same)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=same)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (sta

In [ ]:
feat_test=feat_test.to('cuda')

In [ ]:
  predictions_status = []
  predictions_direction = []
  predictions_model=[]


  for feat_idx in test_loader:
    feat_idx=feat_idx.to(device)
    outputs_status, outputs_direction, outputs_model = model(feat_idx)


    _, predicted_status = torch.max(outputs_status, 1)
    _, predicted_direction = torch.max(outputs_direction, 1)
    _, predicted_model = torch.max(outputs_model, 1)


    predicted_status_cpu = predicted_status.cpu()
    predicted_direction_cpu = predicted_direction.cpu()
    predicted_model_cpu = predicted_model.cpu()


    predicted_status_np = predicted_status_cpu.numpy()
    predicted_direction_np = predicted_direction_cpu.numpy()
    predicted_model_np = predicted_model_cpu.numpy()


  predictions_status.extend(predicted_status_np)
  predictions_direction.extend(predicted_direction_np)
  predictions_model.extend(predicted_model_np)


AttributeError: 'list' object has no attribute 'to'

In [ ]:
df = pd.DataFrame({'fault': predictions_status, 'maneuvering_direction': predictions_direction,'model_type':predictions_model})
df

,fault,maneuvering_direction,model_type
0,6,2,0
1,0,0,0
2,0,1,0
3,0,2,0
4,7,4,0
...,...,...,...
64769,1,2,2
64770,4,3,2
64771,5,3,2
64772,2,2,2


In [ ]:


    predictions_status = []
    predictions_direction = []
    predictions_model=[]
# Iterate over test data

    # Forward pass
    outputs_status, outputs_direction,outputs_model = model(feat_test)

    # Get predicted labels
    _, predicted_status = torch.max(outputs_status, 1)
    _, predicted_direction = torch.max(outputs_direction, 1)
    _, predicted_model = torch.max(outputs_model, 1)
    # Append predictions to list
    predictions_status.extend((predicted_status.cpu()).numpy())
    predictions_direction.extend((predicted_direction.cpu()).numpy())
    predictions_model.extend((predicted_model.cpu()).numpy())
# Create DataFrame from predictions
    df = pd.DataFrame({'fault': predictions_status, 'maneuvering_direction': predictions_direction,'model_type':predictions_model})



In [ ]:
#inverse transformation of label encoding
mod=df['model_type'].tolist()
mod1=label_encoder2.inverse_transform(mod)
print(mod1)
dir=df['maneuvering_direction'].tolist()
dir1=label_encoder1.inverse_transform(dir)
print(dir1)
faul=df['fault'].tolist()
faul1=label_encoder.inverse_transform(faul)
print(faul1)

['A' 'A' 'A' ... 'C' 'C' 'C']
['L' 'R' 'C' ... 'B' 'CC' 'L']
['PC1' 'MF3' 'MF2' ... 'N' 'N' 'MF4']


In [ ]:
df['model_type']=mod1
df['model_type']

0        A
1        A
2        A
3        A
4        A
        ..
64769    C
64770    C
64771    C
64772    C
64773    C
Name: model_type, Length: 64774, dtype: object

In [ ]:
df['maneuvering_direction']=dir1
df['fault']=faul1

In [ ]:
fil=final_test['file_name'].tolist()
df['ID']=fil
df

In [ ]:
df['ID']=fil
df

,fault,maneuvering_direction,model_type,ID
0,PC1,L,A,A_R_PC2_17_ConstructionSite_104_snr=11.7466147...
1,MF3,R,A,A_B_MF3_881_EoeunHill_369_snr=14.2721269545175...
2,MF2,C,A,A_C_MF2_956_ConstructionSite_493_snr=14.999665...
3,MF1,C,A,A_C_MF1_871_WestDoor_150_snr=14.11904469828211...
4,MF2,R,A,A_R_MF2_117_EoeunHill_258_snr=12.3358923008580...
...,...,...,...,...
64769,MF4,L,C,C_L_MF4_921_DuckPond_408_snr=14.68987604267104...
64770,MF2,B,C,C_B_MF2_822_EoeunHill_722_snr=11.7196268001049...
64771,N,B,C,C_B_N_929_WestDoor_551_snr=14.21089595681281.w...
64772,N,CC,C,C_CC_N_887_EoeunHill_650_snr=12.16161579225678...


In [ ]:
new_order=['ID','model_type','maneuvering_direction','fault']
df = df.reindex(columns=new_order)
df

,ID,model_type,maneuvering_direction,fault
0,A_R_PC2_17_ConstructionSite_104_snr=11.7466147...,A,L,PC1
1,A_B_MF3_881_EoeunHill_369_snr=14.2721269545175...,A,R,MF3
2,A_C_MF2_956_ConstructionSite_493_snr=14.999665...,A,C,MF2
3,A_C_MF1_871_WestDoor_150_snr=14.11904469828211...,A,C,MF1
4,A_R_MF2_117_EoeunHill_258_snr=12.3358923008580...,A,R,MF2
...,...,...,...,...
64769,C_L_MF4_921_DuckPond_408_snr=14.68987604267104...,C,L,MF4
64770,C_B_MF2_822_EoeunHill_722_snr=11.7196268001049...,C,B,MF2
64771,C_B_N_929_WestDoor_551_snr=14.21089595681281.w...,C,B,N
64772,C_CC_N_887_EoeunHill_650_snr=12.16161579225678...,C,CC,N


In [ ]:
# Save DataFrame to CSV
df.to_csv('predictions8.csv', index=False)